In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 
from subprocess import call
import matplotlib.cm as cm
from matplotlib import rc
import matplotlib as mpl
import seaborn as sns 
from matplotlib.colors import ListedColormap
from ipywidgets import widgets as wgt
from scipy.interpolate import griddata
from matplotlib.colors import LinearSegmentedColormap
import imageio
import os

from ipywidgets import interact, widgets, FloatSlider, FloatText, fixed
from IPython.display import display, clear_output

from scipy.optimize import minimize
mpl.rcParams.update(
    {
        'figure.constrained_layout.use': True,
        'interactive': False,
        "text.usetex": False,  # Use mathtext, not LaTeX
        "font.family": "cmr10",  # Use the Computer modern font
        "mathtext.fontset": "cm",
        "axes.formatter.use_mathtext": True,
        "axes.unicode_minus": False,
    }
)

In [45]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from matplotlib.colors import LinearSegmentedColormap
import imageio

# Define custom colormap
colors = [(1, 0, 0), (1, 1, 1), (0, 0, 1)]  # Red, White, Blue
cmap_name = 'custom_cmap'
custom_cmap = LinearSegmentedColormap.from_list(cmap_name, colors)

# Define ruta del directorio
output_files_dir = 'OUTPUT_FILES/'

# Ruta de la carpeta para guardar las imágenes
output_images_folder = 'output_images/'

# Creamos la carpeta si no existe
if not os.path.exists(output_images_folder):
    os.makedirs(output_images_folder)

# Read grid data
grid_data = np.loadtxt(output_files_dir + 'wavefield_grid_for_dumps.txt')

# Define regular grid
x = grid_data[:, 0]
y = grid_data[:, 1]
x_min, x_max = np.min(x), np.max(x)
y_min, y_max = np.min(y), np.max(y)
xi = np.linspace(x_min, x_max, 1500)
yi = np.linspace(y_min, y_max, 1500)
xi, yi = np.meshgrid(xi, yi)

# Lista para almacenar los nombres de archivo de las imágenes
image_files = []

for i in range(100, 5001, 100):  # Cambio aquí: de 100 en 100
    # Load wavefield data
    filename = f'{output_images_folder}wavefield_{i*1.0e-4:.2f}.png'
    try:
        wavefield_data = np.loadtxt(output_files_dir + f'wavefield000{i:04d}_01.txt')
    except FileNotFoundError:
        print(f'File wavefield000{i:04d}_01.txt not found. Skipping...')
        continue
    
    # Extract z values (pressure)
    pressure = wavefield_data
    
    # Interpolate pressure values onto the regular grid
    zi = griddata((x, y), pressure, (xi, yi), method='linear')
    
    # Plot interpolated values
    plt.figure(figsize=(4, 3))
    plt.imshow(zi, extent=[x_min, x_max, y_min, y_max], cmap=custom_cmap, origin='lower', aspect='auto', vmin=-np.max(np.abs(pressure)), vmax=np.max(np.abs(pressure)))
    plt.xlabel('Distance (km)', fontsize=10)
    plt.ylabel('Depth (km)', fontsize=10)
    plt.title(f'Time: {i*1.0e-4:.2f} s', fontsize=10)  # Formateo a dos decimales
    #plt.colorbar(label='Interpolated Pressure')
    plt.savefig(filename)  # Save figure as PNG
    plt.close()
    
    # Añadir el nombre de archivo a la lista
    image_files.append(filename)

# Crear GIF animado
with imageio.get_writer('wavefield_animation.gif', mode='I', duration=0.5) as writer:
    for filename in image_files:
        image = imageio.imread(filename)
        writer.append_data(image)

# Eliminar archivos de imagen individuales
for filename in set(image_files):
    os.remove(filename)


/tmp/ipykernel_17560/1103437434.py:69: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
